In [3]:
import yfinance as yf
import pandas as pd

stock_symbol = 'AAPL'
start_date = '2021-01-01'
end_date = '2023-01-01'

stock_data = yf.download(stock_symbol, start=start_date, end=end_date)

closing_prices = stock_data['Close']

[*********************100%%**********************]  1 of 1 completed


In [4]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

stdsc = StandardScaler()
scaler = MinMaxScaler(feature_range=(-1, 1)) 

Processing Data

In [21]:
print(closing_prices["2021-01-04"])

cp_np = []
for date, values in closing_prices.items():
    cp_np.append(values)

print(cp_np)

129.41000366210938
[129.41000366210938, 131.00999450683594, 126.5999984741211, 130.9199981689453, 132.0500030517578, 128.97999572753906, 128.8000030517578, 130.88999938964844, 128.91000366210938, 127.13999938964844, 127.83000183105469, 132.02999877929688, 136.8699951171875, 139.07000732421875, 142.9199981689453, 143.16000366210938, 142.05999755859375, 137.08999633789062, 131.9600067138672, 134.13999938964844, 134.99000549316406, 133.94000244140625, 137.38999938964844, 136.75999450683594, 136.91000366210938, 136.00999450683594, 135.38999938964844, 135.1300048828125, 135.3699951171875, 133.19000244140625, 130.83999633789062, 129.7100067138672, 129.8699951171875, 126.0, 125.86000061035156, 125.3499984741211, 120.98999786376953, 121.26000213623047, 127.79000091552734, 125.12000274658203, 122.05999755859375, 120.12999725341797, 121.41999816894531, 116.36000061035156, 121.08999633789062, 119.9800033569336, 121.95999908447266, 121.02999877929688, 123.98999786376953, 125.56999969482422, 124.76

LSTM Model Def

In [5]:
class LSTM(nn.Module):
    def __init__(self, input_size = 1, hidden_layer_size = 32, num_layers = 2, output_size = 1):
        super(LSTM, self).__init__()
        self.input_dim = input_size
        self.hidden_dim = hidden_layer_size
        self.num_layers = num_layers
        self.output_dim = output_size

        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers, batch_first=True, dropout=0)
        self.fc = nn.Linear(self.hidden_dim, self.output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :])
        return out

In [7]:
model = LSTM()

criterion = torch.nn.MSELoss(reduction='sum')

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
import time

num_epochs = 500
hist = np.zeros(num_epochs)
start_time = time.time()
lstm = []
error = 1
tol = 1e-3

for t in range(num_epochs):
    if (error > tol):
        y_train_pred = model(x_train)
        loss = criterion(y_train_pred, y_train_lstm)
        error = abs(loss.item())
        if (t%10 == 0): print("Epoch ", t, "MSE: %.3e" % loss.item())
        if loss.item()<tol:
            print("Epoch ", t, "MSE: %.3e" % loss.item())
            break
        hist[t] = loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    else:
        break